This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/lqr.html).  I recommend having both windows open, side-by-side!


In [ ]:
import numpy as np
from IPython.display import Markdown, display
from pydrake.all import (
    DiscreteTimeLinearQuadraticRegulator,
    LinearQuadraticRegulator,
    ToLatex,
)

# Discrete-time vs continuous-time LQR

Let's compare the two solutions in our simplest of systems: the double integrator.

## Continuous time

In [ ]:
def ct_demo():
    A = np.array([[0, 1], [0, 0]])
    B = np.array([[0], [1]])
    Q = np.identity(2)
    R = np.identity(1)

    K, S = LinearQuadraticRegulator(A, B, Q, R)
    display(Markdown(f"$S = {ToLatex(S)}$\n"))

    eigenvalues, eigenvectors = np.linalg.eig(S)
    display(Markdown(f"$eig(S) = {ToLatex(eigenvalues)}$\n"))


ct_demo()

## Discrete time

In [ ]:
def dt_demo(h=1):
    A = np.array([[1, h], [0, 1]])
    B = np.array([[0], [h]])
    Q = h * np.identity(2)
    R = h * np.identity(1)

    K, S = DiscreteTimeLinearQuadraticRegulator(A, B, Q, R)
    display(Markdown(f"$S = {ToLatex(S)}$\n"))
    eigenvalues, eigenvectors = np.linalg.eig(S)
    display(Markdown(f"$eig(S) = {ToLatex(eigenvalues)}$\n"))


dt_demo(h=1)

There are a few things to notice here.  First, the discrete-time solution converges to the continuous-time solution as the time step goes to zero.  Second, the cost-to-go is always higher in the discrete-time version: you should think of the discrete time as adding an additional constraint that the control decision can only be changed once per time step.  Adding constraints can only increase the total cost.

### Side note: Algebraic solution

In the DP chapter, I was able to give a nice closed-form solution for the continuous-time case:$$S = \begin{bmatrix} \sqrt{3} & 1 \\ 1 & \sqrt{3} \end{bmatrix}.$$  The discrete-time case is not as clean.  Even when $h=1$, the discrete-time Riccati equation, using $$S = \begin{bmatrix} a & b \\ b & c \end{bmatrix},$$ results in three equations: \begin{gather*} b^2 = 1+c \\ 1 + c + bc = a + ac \\ c^2 = 2b + a + ac. \end{gather*}  With a little work, you can reduce this to to a quadratic equation in $b$, with a solution, $b = \frac{1}{4}(1 + \sqrt{21} + \sqrt{2(3+\sqrt{21})}).$  Not so nice! 

### Side note: Exact integration

One can also use the exact integral of the linear system $e^Ah$ in the discretization, instead of the Euler discretization.  It doesn't change the basic observation.